In [1]:
import sys
import os
import cv2 as cv
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from statistics import mode
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score
from heapq import * 
import time
import copy

#Pytorch
import torch
import torchvision
# import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import LambdaLR
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from sklearn.model_selection import KFold
from scipy import stats
from sklearn.metrics import f1_score, confusion_matrix

import pytorch_model_summary as pms
from torchviz import make_dot
np.random.seed(42)

# import tensorflow as tf
NUM_CLASS = 2
WINDOW = 5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
class MyDataset(Dataset):
    def __init__(self, root_dir=None, transform=None):
        path = os.path.join(root_dir,'labels.csv')
        self.ylabels = pd.read_csv(path)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.ylabels)

    def __getitem__(self, idx):
        seq_len = 5
        if torch.is_tensor(idx):
            idx = idx.tolist()
 
        label = self.ylabels.iloc[idx,1]
        label_str = str(label) + '/'
        img_folder = os.path.join(self.root_dir, label_str)
        
        img_arr = np.zeros((5, 1, 64, 64))
        for i in range (5):
            img_name = img_folder + str(idx) + '-' + str(i) + '.jpg'
            image = io.imread(img_name)
            if self.transform:
                image = self.transform(image)
            img_arr[i][0] = image
            
        label_1hot = np.zeros(NUM_CLASS)
        label_1hot[label] = 1.0
        label_1hot = torch.Tensor(label_1hot)
#         label = np.array([label])
        label = torch.LongTensor(np.array([label]))
        sample = {'images': img_arr, 'label': label_1hot}
        return sample

In [3]:
class Reshape1(torch.nn.Module):
    def forward(self, x):
        batch_size = torch.Size([1,5,1,64,64])
        return x.view(batch_size, -1)
    
class Reshape2(torch.nn.Module):
    def forward(self, x):
        batch_size = torch.Size([1,1,64,64])
        return x.view(batch_size, -1)

def build_encoder():
    encoder = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=192, kernel_size=3),
        nn.BatchNorm2d(num_features=192),
        nn.ReLU(),
        nn.Conv2d(in_channels=192, out_channels=192, kernel_size=1),
        nn.ReLU(),
        
        nn.Conv2d(in_channels=192, out_channels=192, kernel_size=3, stride=2),
        nn.BatchNorm2d(num_features=192),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        
        nn.Conv2d(in_channels=192, out_channels=96, kernel_size=3),
        nn.BatchNorm2d(num_features=96),
        nn.ReLU(),
        nn.Conv2d(in_channels=96, out_channels=96, kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=96, out_channels=96, kernel_size=1),
        nn.ReLU(),
        
#         nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(in_channels=96, out_channels=96, kernel_size=3, stride=2),
        nn.Dropout(p=0.3),
        
        nn.Conv2d(in_channels=96, out_channels=32, kernel_size=3),
        nn.BatchNorm2d(num_features=32),
        nn.ReLU(),
        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3),
        nn.BatchNorm2d(num_features=32),
        nn.ReLU(),
#         nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=2),
#         nn.BatchNorm2d(num_features=32),
#         nn.ReLU()
    )
    return encoder

def build_decoder():
    decoder = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=NUM_CLASS, kernel_size=1),
        nn.AdaptiveAvgPool2d((1,1)),
        nn.Softmax(dim=1)
    )
    return decoder

In [4]:
class Net(nn.Module):
    def __init__(self, encoder, decoder):
        super(Net, self).__init__()
        self.in_encoder = encoder
        self.in_decoder = decoder
        
    def forward(self, x):
        tmp = self.in_encoder(x)
        y = self.in_decoder(tmp)
        return y
    def encoder(self, x):
        return self.in_encoder(x)

In [5]:
def check_validation(model, valloader, loss_function):
    # Evaluationfor this fold
    correct, total = 0, 0
    current_loss = 0.0
    with torch.no_grad():
        # Init the neural network
        encoder = build_encoder()
        decoder = build_decoder()
        trained_net = Net(encoder, decoder)
        trained_net.load_state_dict(model)
        trained_net.to(device)

        # Iterate over the test data and generate predictions
        for i, data in enumerate(valloader, 0):

            # Get inputs
            inputs = data['images']
            targets = data['label']

            outputs = None
            for index, xbatch in enumerate(inputs):

                xbatch = xbatch.float().to(device)
                output = sliding_window(xbatch, trained_net)

                # Perform forward pass
#                 output = trained_net(xbatch)
                if index == 0:
                    outputs = output
                else:
                    outputs = torch.vstack([outputs, output])

            # Compute loss
            ybatches = targets.to(device)
            outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))
            ybatches.squeeze_(1)
            loss = loss_function(outputs, ybatches)
            current_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            _, truth = torch.max(ybatches, 1)
            total += targets.size(0)
            correct += (predicted == truth).sum().item()
    val_loss = current_loss/(i+1)
    val_acc = 100.0 * correct / total
    return val_loss, val_acc

In [6]:
def sliding_window(x, network):         
    outputs = network(x)
    outputs = torch.reshape(outputs,(-1,2))
    final_out = torch.mean(outputs, 0)
    return final_out

In [19]:
# encoder = build_encoder()
# decoder = build_decoder()
# net = Net(encoder, decoder)
# pms.summary(net, 
#     torch.zeros((1,1,64,64)), 
#     batch_size=1, 
#     show_hierarchical=True, 
#     print_summary=True)

In [7]:
def train():
    #---------------Config---------------
    torch.manual_seed(42)
    num_epochs = 50
    k_folds = 5
    patience = 10
    batch_size = 32
    #---------------Config---------------
    
    # Preparation
    weight = torch.tensor([1,1]).to(device)
    
    loss_function = nn.BCELoss(weight=weight)

    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
    ])

    train_data_dir = './Prepared_Data/Train/'
    train_dataset = MyDataset(root_dir=train_data_dir, transform = transformer)

    val_data_dir = './Prepared_Data/Validation/'
    val_dataset = MyDataset(root_dir=val_data_dir, transform = transformer)

    dataset = ConcatDataset([train_dataset, val_dataset])

    kfold = KFold(n_splits=k_folds, shuffle=True)
    
    results = {}

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        training_details = []
        best_model = None

        # Print
        print(f'FOLD {fold}')
        print('--------------------------------')

        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)

        # Define data loaders for training and testing data in this fold
        trainloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=train_subsampler)
        valloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=val_subsampler)

        # Init the neural network
        encoder = build_encoder()
        decoder = build_decoder()
        network = Net(encoder, decoder)
        network.to(device) #use GPU
        use_gpu = True

        # Initialize optimizer
        optimizer = optim.Adam(network.parameters(), lr=0.001)
        max_train_acc, max_val_acc = 0, 0
        early_stop_count = 0
        # Run the training loop for defined number of epochs
        for epoch in range(0, num_epochs):

            correct, total = 0, 0
            # Print epoch
            print(f'Starting epoch {epoch+1}')
            # Set current loss value
            current_loss = 0.0
            epoch_loss = 0.0
            # Iterate over the DataLoader for training data
            for i, data in enumerate(trainloader, 0):
            # Starting 1 cycle of mini-batch

                # Get inputs
                inputs = data['images']
                targets = data['label']

                outputs = None
                for index, xbatch in enumerate(inputs):
                    # Zero the gradients
                    optimizer.zero_grad()

                    xbatch = xbatch.float().to(device)
                    output = sliding_window(xbatch, network)

                    # Perform forward pass
                    if index == 0:
                        outputs = output
                    else:
                        outputs = torch.vstack([outputs, output])

                # Compute loss
                ybatches = targets.to(device)
                outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))
                loss = loss_function(outputs, ybatches)

                # Perform backward pass
                loss.backward()

                # Perform optimization
                optimizer.step()

                # Print statistics
                current_loss += loss.item()
                epoch_loss += loss.item()

                # Compute Accuracy
                _, predicted = torch.max(outputs, 1)
                _, truth = torch.max(ybatches, 1)
                total += targets.size(0)
                correct += (predicted == truth).sum().item()

            epoch_loss = epoch_loss/(i+1)
            epoch_acc = 100.0 * correct / total
            print('Train Loss for Epoch %d: %.4f ' % (epoch, epoch_loss))
            print('Train Accuracy for Epoch %d: %.2f %%' % (epoch, epoch_acc))
            val_loss, val_acc = check_validation(network.state_dict(), valloader, loss_function)
            print('Val Loss for Epoch %d: %.4f ' % (epoch, val_loss))
            print('Val Accuracy for Epoch %d: %.2f %%' % (epoch, val_acc))
            print('--------------------------------')

            training_details.append([epoch_acc, epoch_loss, val_acc, val_loss])

            stop_improving = False
            if val_acc > max_val_acc:
                max_val_acc = val_acc
                early_stop_count = 0
            else:
                early_stop_count += 1
                stop_improving = True
                print("Early Stop Count Increased")

            if epoch_acc > max_train_acc and not stop_improving:
                max_train_acc = copy.deepcopy(epoch_acc)
                best_model = copy.deepcopy(network.state_dict())
                print("Found better model...saving")
            
            print("Early Stop Count", early_stop_count)
            if early_stop_count >= patience:
                print("Early stopping at Epoch", epoch)
                break


        # Process is complete.
        print('Training process has finished. Saving trained model.')

        # Print about testing
        print('Starting testing')

        # Saving the model
        save_path = f'./models/model-fold-{fold}.pth'
        save_path_csv = f'./models/model-fold-{fold}.csv'
        torch.save(best_model, save_path)

        training_details = np.array(training_details)
        df = {'epoch acc': training_details[:,0], 
              'epoch loss': training_details[:,1], 
              'val acc': training_details[:,2], 
              'val loss': training_details[:,3], }
        pd.DataFrame(df).to_csv(save_path_csv)

        # Evaluationfor this fold
        correct, total = 0, 0
        with torch.no_grad():
            val_loss, val_acc = check_validation(torch.load(save_path), valloader, loss_function)
            # Print accuracy
            print('Accuracy for fold %d: %.2f %%' % (fold, val_acc))
            print('--------------------------------')
            results[fold] = val_acc

    # Print fold results
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value} %')
        sum += value
    print(f'Average: {sum/len(results.items())} %')

In [8]:
train()

FOLD 0
--------------------------------
Starting epoch 1
Train Loss for Epoch 0: 0.6944 
Train Accuracy for Epoch 0: 54.47 %
Val Loss for Epoch 0: 0.6823 
Val Accuracy for Epoch 0: 82.22 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 2
Train Loss for Epoch 1: 0.6751 
Train Accuracy for Epoch 1: 77.09 %
Val Loss for Epoch 1: 0.6571 
Val Accuracy for Epoch 1: 78.89 %
--------------------------------
Early Stop Count Increased
Early Stop Count 1
Starting epoch 3
Train Loss for Epoch 2: 0.6551 
Train Accuracy for Epoch 2: 78.21 %
Val Loss for Epoch 2: 0.6389 
Val Accuracy for Epoch 2: 80.00 %
--------------------------------
Early Stop Count Increased
Early Stop Count 2
Starting epoch 4
Train Loss for Epoch 3: 0.6391 
Train Accuracy for Epoch 3: 80.45 %
Val Loss for Epoch 3: 0.6187 
Val Accuracy for Epoch 3: 85.56 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 5
Train Loss for Epoch 4: 0.618

Train Loss for Epoch 5: 0.6161 
Train Accuracy for Epoch 5: 82.12 %
Val Loss for Epoch 5: 0.6484 
Val Accuracy for Epoch 5: 70.00 %
--------------------------------
Early Stop Count Increased
Early Stop Count 4
Starting epoch 7
Train Loss for Epoch 6: 0.6213 
Train Accuracy for Epoch 6: 74.58 %
Val Loss for Epoch 6: 0.6422 
Val Accuracy for Epoch 6: 74.44 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 8
Train Loss for Epoch 7: 0.5808 
Train Accuracy for Epoch 7: 84.08 %
Val Loss for Epoch 7: 0.6307 
Val Accuracy for Epoch 7: 73.33 %
--------------------------------
Early Stop Count Increased
Early Stop Count 1
Starting epoch 9
Train Loss for Epoch 8: 0.5568 
Train Accuracy for Epoch 8: 84.92 %
Val Loss for Epoch 8: 0.6068 
Val Accuracy for Epoch 8: 75.56 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 10
Train Loss for Epoch 9: 0.5083 
Train Accuracy for Epoch 9: 87.99 %
Val Loss for Epoc

Train Loss for Epoch 0: 0.6943 
Train Accuracy for Epoch 0: 51.12 %
Val Loss for Epoch 0: 0.6870 
Val Accuracy for Epoch 0: 47.78 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 2
Train Loss for Epoch 1: 0.6730 
Train Accuracy for Epoch 1: 62.85 %
Val Loss for Epoch 1: 0.6711 
Val Accuracy for Epoch 1: 63.33 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 3
Train Loss for Epoch 2: 0.6552 
Train Accuracy for Epoch 2: 75.42 %
Val Loss for Epoch 2: 0.6566 
Val Accuracy for Epoch 2: 66.67 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 4
Train Loss for Epoch 3: 0.6370 
Train Accuracy for Epoch 3: 75.42 %
Val Loss for Epoch 3: 0.6412 
Val Accuracy for Epoch 3: 70.00 %
--------------------------------
Early Stop Count 0
Starting epoch 5
Train Loss for Epoch 4: 0.6197 
Train Accuracy for Epoch 4: 78.21 %
Val Loss for Epoch 4: 0.6202 
Val Accuracy f

Train Loss for Epoch 14: 0.3620 
Train Accuracy for Epoch 14: 93.31 %
Val Loss for Epoch 14: 0.3669 
Val Accuracy for Epoch 14: 92.13 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 16
Train Loss for Epoch 15: 0.3396 
Train Accuracy for Epoch 15: 90.53 %
Val Loss for Epoch 15: 0.3542 
Val Accuracy for Epoch 15: 92.13 %
--------------------------------
Early Stop Count Increased
Early Stop Count 1
Starting epoch 17
Train Loss for Epoch 16: 0.3252 
Train Accuracy for Epoch 16: 93.04 %
Val Loss for Epoch 16: 0.4452 
Val Accuracy for Epoch 16: 79.78 %
--------------------------------
Early Stop Count Increased
Early Stop Count 2
Starting epoch 18
Train Loss for Epoch 17: 0.3671 
Train Accuracy for Epoch 17: 86.91 %
Val Loss for Epoch 17: 0.3629 
Val Accuracy for Epoch 17: 87.64 %
--------------------------------
Early Stop Count Increased
Early Stop Count 3
Starting epoch 19
Train Loss for Epoch 18: 0.2885 
Train Accuracy for Epoch 18: 93.04

Train Loss for Epoch 16: 0.2871 
Train Accuracy for Epoch 16: 92.76 %
Val Loss for Epoch 16: 0.2694 
Val Accuracy for Epoch 16: 93.26 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 18
Train Loss for Epoch 17: 0.3135 
Train Accuracy for Epoch 17: 90.53 %
Val Loss for Epoch 17: 0.2725 
Val Accuracy for Epoch 17: 93.26 %
--------------------------------
Early Stop Count Increased
Early Stop Count 1
Starting epoch 19
Train Loss for Epoch 18: 0.2673 
Train Accuracy for Epoch 18: 91.92 %
Val Loss for Epoch 18: 0.3250 
Val Accuracy for Epoch 18: 88.76 %
--------------------------------
Early Stop Count Increased
Early Stop Count 2
Starting epoch 20
Train Loss for Epoch 19: 0.2419 
Train Accuracy for Epoch 19: 94.71 %
Val Loss for Epoch 19: 0.2829 
Val Accuracy for Epoch 19: 89.89 %
--------------------------------
Early Stop Count Increased
Early Stop Count 3
Starting epoch 21
Train Loss for Epoch 20: 0.2227 
Train Accuracy for Epoch 20: 94.43

In [7]:
def evaluate():
    avg_acc = 0
    avg_f1 = 0
    avg_conf = np.zeros((2,2))
    
    # Preparation
    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
    ])
    test_data_dir = './Prepared_Data/Test/'
    test_dataset = MyDataset(root_dir=test_data_dir, transform = transformer)

    for i in range(5):
        PATH = './Plan3_Models/best/model-fold-' + str(i) + '.pth'
        encoder = build_encoder()
        decoder = build_decoder()
        trained_net = Net(encoder,decoder)
        trained_net.load_state_dict(torch.load(PATH))
        trained_net.to(device)

        correct = 0
        total = 0
        y_true, y_pred = [], []
        testloader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size = 50)

        y_pred = []
        y_true = []
        with torch.no_grad():
            # Iterate over the test data and generate predictions
            for i, data in enumerate(testloader, 0):

                # Get inputs
                inputs = data['images']
                targets = data['label']

                outputs = None
                for index, xbatch in enumerate(inputs):

                    xbatch = xbatch.float().to(device)

                    # Perform forward pass
                    output = sliding_window(xbatch, trained_net)
                    if index == 0:
                        outputs = output
                    else:
                        outputs = torch.vstack([outputs, output])

                # Compute loss
                ybatches = targets.to(device)
                outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))

                _, predicted = torch.max(outputs, 1)
                _, truth = torch.max(ybatches, 1)
                total += targets.size(0)
                correct += (predicted == truth).sum().item()

                y_pred.extend(predicted.cpu())
                y_true.extend(truth.cpu())

        # Print accuracy
        print("Accuracy", 100.0 * correct / total)
        f1 = f1_score(y_true, y_pred, labels=[0,1], average = 'binary')
        confusion_matr = confusion_matrix(y_true, y_pred, labels=[0,1])
        print("F1", f1)
        print(confusion_matr)

        avg_acc += 100.0 * correct / total
        avg_f1 += f1
        np.add(avg_conf, np.array(confusion_matr))

    avg_acc = avg_acc/5
    avg_f1 = avg_f1/5
    np.true_divide(avg_conf, 5)
    print("--------------Average------------")
    print('Avg Accuracy', avg_acc)
    print('Avg F1', avg_f1)
    print('Avg Confusion', avg_conf)


In [10]:
evaluate()

Accuracy 93.75
F1 0.9393939393939394
[[29  3]
 [ 1 31]]
Accuracy 81.25
F1 0.8421052631578948
[[20 12]
 [ 0 32]]
Accuracy 89.0625
F1 0.8955223880597014
[[27  5]
 [ 2 30]]
Accuracy 84.375
F1 0.8387096774193549
[[28  4]
 [ 6 26]]
Accuracy 90.625
F1 0.911764705882353
[[27  5]
 [ 1 31]]
--------------Average------------
Avg Accuracy 87.8125
Avg F1 0.8854991947826487
Avg Confusion [[0. 0.]
 [0. 0.]]


In [10]:
class DomainADDataset(Dataset):
    def __init__(self, root_dir=None, transform=None):
        path = os.path.join(root_dir,'labels.csv')
        self.ylabels = pd.read_csv(path)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.ylabels)

    def __getitem__(self, idx):
        seq_len = 5
        if torch.is_tensor(idx):
            idx = idx.tolist()
 
        label = self.ylabels.iloc[idx]['Label']
        parti_id = self.ylabels.iloc[idx]['ID']
        label_str = str(label) + '/'
        img_folder = os.path.join(self.root_dir, label_str)
        
        img_arr = np.zeros((5, 1, 64, 64))
        for i in range (5):
            img_name = img_folder + str(parti_id) + '-' + str(i) + '.jpg'
            image = io.imread(img_name)
            if self.transform:
                image = self.transform(image)
            img_arr[i][0] = image
            
        label_1hot = np.zeros(NUM_CLASS)
        label_1hot[label] = 1.0
        label_1hot = torch.Tensor(label_1hot)
#         label = np.array([label])
        label = torch.LongTensor(np.array([label]))
        sample = {'images': img_arr, 'label': label_1hot}
        return sample

### Supervised

In [9]:
def train_tf():
    #---------------Config---------------
    num_epochs = 50
    k_folds = 5
    patience = 5
    batch_size = 1
    PATH = './Plan3_Models/best/model-fold-' + str(0) + '.pth'
    #---------------Config---------------
    
    # Preparation
    weight = torch.tensor([1,1.03]).to(device)
    
    loss_function = nn.BCELoss(weight=weight)

    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
    ])
    
    train_data_dir = './SelfLabelEngagement_Prepared/Train/'
    train_dataset = DomainADDataset(root_dir=train_data_dir, transform = transformer)

    val_data_dir = './SelfLabelEngagement_Prepared/Validation/'
    val_dataset = DomainADDataset(root_dir=val_data_dir, transform = transformer)

    dataset = ConcatDataset([train_dataset, val_dataset])

    kfold = KFold(n_splits=k_folds, shuffle=True)
    
    results = {}

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        
        training_details = []
        best_model = None

        # Print
        print(f'FOLD {fold}')
        print('--------------------------------')

        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)

        # Define data loaders for training and testing data in this fold
        trainloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=train_subsampler)
        valloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=val_subsampler)

        # Init the neural network
        encoder = build_encoder()
        decoder = build_decoder()
        network = Net(encoder, decoder)
        network.load_state_dict(torch.load(PATH))
        for param in network.in_encoder.parameters():
            param.requires_grad = False
        network.to(device) #use GPU
        use_gpu = True

        # Initialize optimizer
        optimizer = optim.Adam(network.parameters(), lr=0.0005)
        max_train_acc, max_val_acc = 0, 0
        early_stop_count = 0
        # Run the training loop for defined number of epochs
        for epoch in range(0, num_epochs):

            correct, total = 0, 0
            # Print epoch
            print(f'Starting epoch {epoch+1}')
            # Set current loss value
            current_loss = 0.0
            epoch_loss = 0.0
            # Iterate over the DataLoader for training data
            for i, data in enumerate(trainloader, 0):
            # Starting 1 cycle of mini-batch

                # Get inputs
                # Get inputs
                inputs = data['images']
                targets = data['label']

                outputs = None
                for index, xbatch in enumerate(inputs):
                    # Zero the gradients
                    optimizer.zero_grad()

                    xbatch = xbatch.float().to(device)
                    output = sliding_window(xbatch, network)

                    # Perform forward pass
                    if index == 0:
                        outputs = output
                    else:
                        outputs = torch.vstack([outputs, output])

                # Compute loss
                ybatches = targets.to(device)
                outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))
                loss = loss_function(outputs, ybatches)

                # Perform backward pass
                loss.backward()

                # Perform optimization
                optimizer.step()

                # Print statistics
                current_loss += loss.item()
                epoch_loss += loss.item()

                # Compute Accuracy
                _, predicted = torch.max(outputs, 1)
                _, truth = torch.max(ybatches, 1)
                total += targets.size(0)
                correct += (predicted == truth).sum().item()

            epoch_loss = epoch_loss/(i+1)
            epoch_acc = 100.0 * correct / total
            print('Train Loss for Epoch %d: %.4f ' % (epoch, epoch_loss))
            print('Train Accuracy for Epoch %d: %.2f %%' % (epoch, epoch_acc))
            val_loss, val_acc = check_validation(network.state_dict(), valloader, loss_function)
            print('Val Loss for Epoch %d: %.4f ' % (epoch, val_loss))
            print('Val Accuracy for Epoch %d: %.2f %%' % (epoch, val_acc))
            print('--------------------------------')

            training_details.append([epoch_acc, epoch_loss, val_acc, val_loss])

            stop_improving = False
            if val_acc > max_val_acc:
                max_val_acc = val_acc
                early_stop_count = 0
            else:
                early_stop_count += 1
                stop_improving = True
                print("Early Stop Count Increased")

            if epoch_acc > max_train_acc and not stop_improving:
                max_train_acc = copy.deepcopy(epoch_acc)
                best_model = copy.deepcopy(network.state_dict())
                print("Found better model...saving")
            
            print("Early Stop Count", early_stop_count)
            if early_stop_count >= patience:
                print("Early stopping at Epoch", epoch)
                break


        # Process is complete.
        print('Training process has finished. Saving trained model.')

        # Print about testing
        print('Starting testing')

        # Saving the model
        save_path = f'./models/model-fold-{fold}.pth'
        save_path_csv = f'./models/model-fold-{fold}.csv'
        torch.save(best_model, save_path)

        training_details = np.array(training_details)
        df = {'epoch acc': training_details[:,0], 
              'epoch loss': training_details[:,1], 
              'val acc': training_details[:,2], 
              'val loss': training_details[:,3], }
        pd.DataFrame(df).to_csv(save_path_csv)

        # Evaluationfor this fold
        correct, total = 0, 0
        with torch.no_grad():
            val_loss, val_acc = check_validation(torch.load(save_path), valloader, loss_function)
            # Print accuracy
            print('Accuracy for fold %d: %.2f %%' % (fold, val_acc))
            print('--------------------------------')
            results[fold] = val_acc

    # Print fold results
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value} %')
        sum += value
    print(f'Average: {sum/len(results.items())} %')

In [12]:
torch.manual_seed(32)
train_tf()

FOLD 0
--------------------------------
Starting epoch 1
Train Loss for Epoch 0: 0.7342 
Train Accuracy for Epoch 0: 40.29 %
Val Loss for Epoch 0: 0.7032 
Val Accuracy for Epoch 0: 60.00 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 2
Train Loss for Epoch 1: 0.7225 
Train Accuracy for Epoch 1: 45.32 %
Val Loss for Epoch 1: 0.6985 
Val Accuracy for Epoch 1: 51.43 %
--------------------------------
Early Stop Count Increased
Early Stop Count 1
Starting epoch 3
Train Loss for Epoch 2: 0.7164 
Train Accuracy for Epoch 2: 50.36 %
Val Loss for Epoch 2: 0.7236 
Val Accuracy for Epoch 2: 42.86 %
--------------------------------
Early Stop Count Increased
Early Stop Count 2
Starting epoch 4
Train Loss for Epoch 3: 0.7207 
Train Accuracy for Epoch 3: 49.64 %
Val Loss for Epoch 3: 0.6922 
Val Accuracy for Epoch 3: 51.43 %
--------------------------------
Early Stop Count Increased
Early Stop Count 3
Starting epoch 5
Train Loss for Epoch 4: 0.7210

Train Loss for Epoch 5: 0.7158 
Train Accuracy for Epoch 5: 48.92 %
Val Loss for Epoch 5: 0.7526 
Val Accuracy for Epoch 5: 37.14 %
--------------------------------
Early Stop Count Increased
Early Stop Count 2
Starting epoch 7
Train Loss for Epoch 6: 0.7093 
Train Accuracy for Epoch 6: 56.12 %
Val Loss for Epoch 6: 0.7527 
Val Accuracy for Epoch 6: 45.71 %
--------------------------------
Early Stop Count Increased
Early Stop Count 3
Starting epoch 8
Train Loss for Epoch 7: 0.7109 
Train Accuracy for Epoch 7: 53.24 %
Val Loss for Epoch 7: 0.7440 
Val Accuracy for Epoch 7: 42.86 %
--------------------------------
Early Stop Count Increased
Early Stop Count 4
Starting epoch 9
Train Loss for Epoch 8: 0.7111 
Train Accuracy for Epoch 8: 51.80 %
Val Loss for Epoch 8: 0.7430 
Val Accuracy for Epoch 8: 48.57 %
--------------------------------
Early Stop Count Increased
Early Stop Count 5
Early stopping at Epoch 8
Training process has finished. Saving trained model.
Starting testing
Accuracy 

In [27]:
torch.manual_seed(32)
def evaluate():
    avg_acc = 0
    avg_f1 = 0
    avg_conf = np.zeros((2,2))
    
    # Preparation
    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
    ])
    test_data_dir = './SelfLabelEngagement_Prepared/Test/'
    test_dataset = DomainADDataset(root_dir=test_data_dir, transform = transformer)

    for i in range(5):
        PATH = './TF_Models/best/model-fold-' + str(i) + '.pth'
        encoder = build_encoder()
        decoder = build_decoder()
        trained_net = Net(encoder,decoder)
        trained_net.load_state_dict(torch.load(PATH))
        trained_net.to(device)

        correct = 0
        total = 0
        y_true, y_pred = [], []
        testloader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size = 50)

        y_pred = []
        y_true = []
        with torch.no_grad():
            # Iterate over the test data and generate predictions
            for i, data in enumerate(testloader, 0):

                # Get inputs
                inputs = data['images']
                targets = data['label']

                outputs = None
                for index, xbatch in enumerate(inputs):

                    xbatch = xbatch.float().to(device)

                    # Perform forward pass
                    output = sliding_window(xbatch, trained_net)
                    if index == 0:
                        outputs = output
                    else:
                        outputs = torch.vstack([outputs, output])

                # Compute loss
                ybatches = targets.to(device)
                outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))

                _, predicted = torch.max(outputs, 1)
                _, truth = torch.max(ybatches, 1)
                total += targets.size(0)
                correct += (predicted == truth).sum().item()

                y_pred.extend(predicted.cpu())
                y_true.extend(truth.cpu())

        # Print accuracy
        print("Accuracy", 100.0 * correct / total)
        f1 = f1_score(y_true, y_pred, labels=[0,1], average = 'binary')
        confusion_matr = confusion_matrix(y_true, y_pred, labels=[0,1])
        print("F1", f1)
        print(confusion_matr)

        avg_acc += 100.0 * correct / total
        avg_f1 += f1
        np.add(avg_conf, np.array(confusion_matr))

    avg_acc = avg_acc/5
    avg_f1 = avg_f1/5
    np.true_divide(avg_conf, 5)
    print("--------------Average------------")
    print('Avg Accuracy', avg_acc)
    print('Avg F1', avg_f1)
    print('Avg Confusion', avg_conf)


### Semi - Supervised

In [7]:
def semi_train_tf():
    #---------------Config---------------
    num_epochs = 50
    k_folds = 5
    patience = 5
    batch_size = 1
    PATH = './Plan3_Models/best/model-fold-' + str(0) + '.pth'
    #---------------Config---------------
    
    # Preparation
    weight = torch.tensor([1,1.03]).to(device)
    
    loss_function = nn.BCELoss(weight=weight)

    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
    ])
    
    train_data_dir = './SelfLabelEngagement_Prepared2/Train/'
    train_dataset = DomainADDataset(root_dir=train_data_dir, transform = transformer)

    val_data_dir = './SelfLabelEngagement_Prepared2/Validation/'
    val_dataset = DomainADDataset(root_dir=val_data_dir, transform = transformer)

    dataset = ConcatDataset([train_dataset, val_dataset])

    kfold = KFold(n_splits=k_folds, shuffle=True)
    
    results = {}

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
        
        training_details = []
        best_model = None

        # Print
        print(f'FOLD {fold}')
        print('--------------------------------')

        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)

        # Define data loaders for training and testing data in this fold
        trainloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=train_subsampler)
        valloader = torch.utils.data.DataLoader(
                        dataset,
                        batch_size=batch_size, sampler=val_subsampler)

        # Init the neural network
        encoder = build_encoder()
        decoder = build_decoder()
        network = Net(encoder, decoder)
        network.load_state_dict(torch.load(PATH))
        for param in network.in_encoder.parameters():
            param.requires_grad = False
        network.to(device) #use GPU
        use_gpu = True

        # Initialize optimizer
        optimizer = optim.Adam(network.parameters(), lr=0.0005)
        max_train_acc, max_val_acc = 0, 0
        early_stop_count = 0
        # Run the training loop for defined number of epochs
        for epoch in range(0, num_epochs):

            correct, total = 0, 0
            # Print epoch
            print(f'Starting epoch {epoch+1}')
            # Set current loss value
            current_loss = 0.0
            epoch_loss = 0.0
            # Iterate over the DataLoader for training data
            for i, data in enumerate(trainloader, 0):
            # Starting 1 cycle of mini-batch

                # Get inputs
                # Get inputs
                inputs = data['images']
                targets = data['label']

                outputs = None
                for index, xbatch in enumerate(inputs):
                    # Zero the gradients
                    optimizer.zero_grad()

                    xbatch = xbatch.float().to(device)
                    output = sliding_window(xbatch, network)

                    # Perform forward pass
                    if index == 0:
                        outputs = output
                    else:
                        outputs = torch.vstack([outputs, output])

                # Compute loss
                ybatches = targets.to(device)
                outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))
                loss = loss_function(outputs, ybatches)

                # Perform backward pass
                loss.backward()

                # Perform optimization
                optimizer.step()

                # Print statistics
                current_loss += loss.item()
                epoch_loss += loss.item()

                # Compute Accuracy
                _, predicted = torch.max(outputs, 1)
                _, truth = torch.max(ybatches, 1)
                total += targets.size(0)
                correct += (predicted == truth).sum().item()

            epoch_loss = epoch_loss/(i+1)
            epoch_acc = 100.0 * correct / total
            print('Train Loss for Epoch %d: %.4f ' % (epoch, epoch_loss))
            print('Train Accuracy for Epoch %d: %.2f %%' % (epoch, epoch_acc))
            val_loss, val_acc = check_validation(network.state_dict(), valloader, loss_function)
            print('Val Loss for Epoch %d: %.4f ' % (epoch, val_loss))
            print('Val Accuracy for Epoch %d: %.2f %%' % (epoch, val_acc))
            print('--------------------------------')

            training_details.append([epoch_acc, epoch_loss, val_acc, val_loss])

            stop_improving = False
            if val_acc > max_val_acc:
                max_val_acc = val_acc
                early_stop_count = 0
            else:
                early_stop_count += 1
                stop_improving = True
                print("Early Stop Count Increased")

            if epoch_acc > max_train_acc and not stop_improving:
                max_train_acc = copy.deepcopy(epoch_acc)
                best_model = copy.deepcopy(network.state_dict())
                print("Found better model...saving")
            
            print("Early Stop Count", early_stop_count)
            if early_stop_count >= patience:
                print("Early stopping at Epoch", epoch)
                break


        # Process is complete.
        print('Training process has finished. Saving trained model.')

        # Print about testing
        print('Starting testing')

        # Saving the model
        save_path = f'./models/model-fold-{fold}.pth'
        save_path_csv = f'./models/model-fold-{fold}.csv'
        torch.save(best_model, save_path)

        training_details = np.array(training_details)
        df = {'epoch acc': training_details[:,0], 
              'epoch loss': training_details[:,1], 
              'val acc': training_details[:,2], 
              'val loss': training_details[:,3], }
        pd.DataFrame(df).to_csv(save_path_csv)

        # Evaluationfor this fold
        correct, total = 0, 0
        with torch.no_grad():
            val_loss, val_acc = check_validation(torch.load(save_path), valloader, loss_function)
            # Print accuracy
            print('Accuracy for fold %d: %.2f %%' % (fold, val_acc))
            print('--------------------------------')
            results[fold] = val_acc

    # Print fold results
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value} %')
        sum += value
    print(f'Average: {sum/len(results.items())} %')

In [11]:
torch.manual_seed(32)
semi_train_tf()

FOLD 0
--------------------------------
Starting epoch 1
Train Loss for Epoch 0: 0.5993 
Train Accuracy for Epoch 0: 63.48 %
Val Loss for Epoch 0: 0.5906 
Val Accuracy for Epoch 0: 68.97 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 2
Train Loss for Epoch 1: 0.5974 
Train Accuracy for Epoch 1: 66.09 %
Val Loss for Epoch 1: 0.5831 
Val Accuracy for Epoch 1: 68.97 %
--------------------------------
Early Stop Count Increased
Early Stop Count 1
Starting epoch 3
Train Loss for Epoch 2: 0.5925 
Train Accuracy for Epoch 2: 64.78 %
Val Loss for Epoch 2: 0.5849 
Val Accuracy for Epoch 2: 65.52 %
--------------------------------
Early Stop Count Increased
Early Stop Count 2
Starting epoch 4
Train Loss for Epoch 3: 0.5959 
Train Accuracy for Epoch 3: 65.65 %
Val Loss for Epoch 3: 0.5882 
Val Accuracy for Epoch 3: 60.34 %
--------------------------------
Early Stop Count Increased
Early Stop Count 3
Starting epoch 5
Train Loss for Epoch 4: 0.5898

Train Loss for Epoch 0: 0.6009 
Train Accuracy for Epoch 0: 62.77 %
Val Loss for Epoch 0: 0.6000 
Val Accuracy for Epoch 0: 70.18 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 2
Train Loss for Epoch 1: 0.5994 
Train Accuracy for Epoch 1: 61.90 %
Val Loss for Epoch 1: 0.6182 
Val Accuracy for Epoch 1: 63.16 %
--------------------------------
Early Stop Count Increased
Early Stop Count 1
Starting epoch 3
Train Loss for Epoch 2: 0.5903 
Train Accuracy for Epoch 2: 65.37 %
Val Loss for Epoch 2: 0.6093 
Val Accuracy for Epoch 2: 63.16 %
--------------------------------
Early Stop Count Increased
Early Stop Count 2
Starting epoch 4
Train Loss for Epoch 3: 0.5853 
Train Accuracy for Epoch 3: 66.67 %
Val Loss for Epoch 3: 0.6155 
Val Accuracy for Epoch 3: 61.40 %
--------------------------------
Early Stop Count Increased
Early Stop Count 3
Starting epoch 5
Train Loss for Epoch 4: 0.5814 
Train Accuracy for Epoch 4: 67.10 %
Val Loss for Epoch 

In [12]:
def semi_evaluate():
    avg_acc = 0
    avg_f1 = 0
    avg_conf = np.zeros((2,2))
    
    # Preparation
    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
    ])
    test_data_dir = './SelfLabelEngagement_Prepared2/Test/'
    test_dataset = DomainADDataset(root_dir=test_data_dir, transform = transformer)

    for i in range(5):
        PATH = './TF_Models/semi-best/model-fold-' + str(i) + '.pth'
        encoder = build_encoder()
        decoder = build_decoder()
        trained_net = Net(encoder,decoder)
        trained_net.load_state_dict(torch.load(PATH))
        trained_net.to(device)

        correct = 0
        total = 0
        y_true, y_pred = [], []
        testloader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size = 50)

        y_pred = []
        y_true = []
        with torch.no_grad():
            # Iterate over the test data and generate predictions
            for i, data in enumerate(testloader, 0):

                # Get inputs
                inputs = data['images']
                targets = data['label']

                outputs = None
                for index, xbatch in enumerate(inputs):

                    xbatch = xbatch.float().to(device)

                    # Perform forward pass
                    output = sliding_window(xbatch, trained_net)
                    if index == 0:
                        outputs = output
                    else:
                        outputs = torch.vstack([outputs, output])

                # Compute loss
                ybatches = targets.to(device)
                outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))

                _, predicted = torch.max(outputs, 1)
                _, truth = torch.max(ybatches, 1)
                total += targets.size(0)
                correct += (predicted == truth).sum().item()

                y_pred.extend(predicted.cpu())
                y_true.extend(truth.cpu())

        # Print accuracy
        print("Accuracy", 100.0 * correct / total)
        f1 = f1_score(y_true, y_pred, labels=[0,1], average = 'binary')
        confusion_matr = confusion_matrix(y_true, y_pred, labels=[0,1])
        print("F1", f1)
        print(confusion_matr)

        avg_acc += 100.0 * correct / total
        avg_f1 += f1
        np.add(avg_conf, np.array(confusion_matr))

    avg_acc = avg_acc/5
    avg_f1 = avg_f1/5
    np.true_divide(avg_conf, 5)
    print("--------------Average------------")
    print('Avg Accuracy', avg_acc)
    print('Avg F1', avg_f1)
    print('Avg Confusion', avg_conf)

In [14]:
semi_evaluate()

Accuracy 45.833333333333336
F1 0.13333333333333333
[[10  2]
 [11  1]]
Accuracy 45.833333333333336
F1 0.13333333333333333
[[10  2]
 [11  1]]
Accuracy 50.0
F1 0.25
[[10  2]
 [10  2]]
Accuracy 45.833333333333336
F1 0.23529411764705882
[[ 9  3]
 [10  2]]
Accuracy 58.333333333333336
F1 0.5833333333333334
[[7 5]
 [5 7]]
--------------Average------------
Avg Accuracy 49.16666666666667
Avg F1 0.2670588235294118
Avg Confusion [[0. 0.]
 [0. 0.]]


### No K-fold

In [12]:
def train_tf_nokfold():
    #---------------Config---------------
    num_epochs = 50
    k_folds = 1
    patience = 10
    batch_size = 1
    PATH = './Plan3_Models/best/model-fold-' + str(0) + '.pth'
    #---------------Config---------------
    
    # Preparation
    weight = torch.tensor([1,1.03]).to(device)
    
    loss_function = nn.BCELoss(weight=weight)

    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5)),
    ])
    
    train_data_dir = './SelfLabelEngagement_Prepared/Train/'
    train_dataset = DomainADDataset(root_dir=train_data_dir, transform = transformer)

    val_data_dir = './SelfLabelEngagement_Prepared/Validation/'
    val_dataset = DomainADDataset(root_dir=val_data_dir, transform = transformer)

#     dataset = ConcatDataset([train_dataset, val_dataset])
#     kfold = KFold(n_splits=k_folds, shuffle=True)
    
    results = {}
    
    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
        train_dataset,
        shuffle = True,
        batch_size=batch_size)
    valloader = torch.utils.data.DataLoader(
        val_dataset,
        shuffle = True,
        batch_size=batch_size)
    

    for fold in range(1):
        
        training_details = []
        best_model = None
        
        print(f'FOLD {fold}')
        print('--------------------------------')

        # Init the neural network
        encoder = build_encoder()
        decoder = build_decoder()
        network = Net(encoder, decoder)
        network.load_state_dict(torch.load(PATH))
        for param in network.in_encoder.parameters():
            param.requires_grad = False
        network.to(device) #use GPU
        use_gpu = True

        # Initialize optimizer
        optimizer = optim.Adam(network.parameters(), lr=0.0001)
        max_train_acc, max_val_acc = 0, 0
        early_stop_count = 0
        # Run the training loop for defined number of epochs
        for epoch in range(0, num_epochs):

            correct, total = 0, 0
            # Print epoch
            print(f'Starting epoch {epoch+1}')
            # Set current loss value
            current_loss = 0.0
            epoch_loss = 0.0
            # Iterate over the DataLoader for training data
            for i, data in enumerate(trainloader, 0):
            # Starting 1 cycle of mini-batch

                # Get inputs
                # Get inputs
                inputs = data['images']
                targets = data['label']

                outputs = None
                for index, xbatch in enumerate(inputs):
                    # Zero the gradients
                    optimizer.zero_grad()

                    xbatch = xbatch.float().to(device)
                    output = sliding_window(xbatch, network)

                    # Perform forward pass
                    if index == 0:
                        outputs = output
                    else:
                        outputs = torch.vstack([outputs, output])

                # Compute loss
                ybatches = targets.to(device)
                outputs = torch.reshape(outputs, (ybatches.shape[0],NUM_CLASS))
                loss = loss_function(outputs, ybatches)

                # Perform backward pass
                loss.backward()

                # Perform optimization
                optimizer.step()

                # Print statistics
                current_loss += loss.item()
                epoch_loss += loss.item()

                # Compute Accuracy
                _, predicted = torch.max(outputs, 1)
                _, truth = torch.max(ybatches, 1)
                total += targets.size(0)
                correct += (predicted == truth).sum().item()

            epoch_loss = epoch_loss/(i+1)
            epoch_acc = 100.0 * correct / total
            print('Train Loss for Epoch %d: %.4f ' % (epoch, epoch_loss))
            print('Train Accuracy for Epoch %d: %.2f %%' % (epoch, epoch_acc))
            val_loss, val_acc = check_validation(network.state_dict(), valloader, loss_function)
            print('Val Loss for Epoch %d: %.4f ' % (epoch, val_loss))
            print('Val Accuracy for Epoch %d: %.2f %%' % (epoch, val_acc))
            print('--------------------------------')

            training_details.append([epoch_acc, epoch_loss, val_acc, val_loss])

            stop_improving = False
            if val_acc > max_val_acc:
                max_val_acc = val_acc
                early_stop_count = 0
            else:
                early_stop_count += 1
                stop_improving = True
                print("Early Stop Count Increased")

            if epoch_acc > max_train_acc and not stop_improving:
                max_train_acc = copy.deepcopy(epoch_acc)
                best_model = copy.deepcopy(network.state_dict())
                print("Found better model...saving")
            
            print("Early Stop Count", early_stop_count)
            if early_stop_count >= patience:
                print("Early stopping at Epoch", epoch)
                break


        # Process is complete.
        print('Training process has finished. Saving trained model.')

        # Print about testing
        print('Starting testing')

        # Saving the model
        save_path = f'./models/model-fold-{fold}.pth'
        save_path_csv = f'./models/model-fold-{fold}.csv'
        torch.save(best_model, save_path)

        training_details = np.array(training_details)
        df = {'epoch acc': training_details[:,0], 
              'epoch loss': training_details[:,1], 
              'val acc': training_details[:,2], 
              'val loss': training_details[:,3], }
        pd.DataFrame(df).to_csv(save_path_csv)

    # Evaluationfor this fold
    correct, total = 0, 0
    with torch.no_grad():
        val_loss, val_acc = check_validation(torch.load(save_path), valloader, loss_function)
        # Print accuracy
        print('Accuracy for fold %d: %.2f %%' % (fold, val_acc))
        print('--------------------------------')
        results[fold] = val_acc

    # Print fold results
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value} %')
        sum += value
    print(f'Average: {sum/len(results.items())} %')

In [13]:
np.random.seed(32)
torch.manual_seed(32)
train_tf_nokfold()

FOLD 0
--------------------------------
Starting epoch 1
Train Loss for Epoch 0: 0.7321 
Train Accuracy for Epoch 0: 45.45 %
Val Loss for Epoch 0: 0.7344 
Val Accuracy for Epoch 0: 40.00 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 2
Train Loss for Epoch 1: 0.7249 
Train Accuracy for Epoch 1: 44.81 %
Val Loss for Epoch 1: 0.7191 
Val Accuracy for Epoch 1: 40.00 %
--------------------------------
Early Stop Count Increased
Early Stop Count 1
Starting epoch 3
Train Loss for Epoch 2: 0.7192 
Train Accuracy for Epoch 2: 44.81 %
Val Loss for Epoch 2: 0.7284 
Val Accuracy for Epoch 2: 45.00 %
--------------------------------
Early Stop Count 0
Starting epoch 4
Train Loss for Epoch 3: 0.7254 
Train Accuracy for Epoch 3: 47.40 %
Val Loss for Epoch 3: 0.7212 
Val Accuracy for Epoch 3: 50.00 %
--------------------------------
Found better model...saving
Early Stop Count 0
Starting epoch 5
Train Loss for Epoch 4: 0.7172 
Train Accuracy for Epoch

In [19]:
evaluate()

Accuracy 93.75
F1 0.9393939393939394
[[29  3]
 [ 1 31]]
Accuracy 79.6875
F1 0.8311688311688311
[[19 13]
 [ 0 32]]
Accuracy 89.0625
F1 0.8955223880597014
[[27  5]
 [ 2 30]]
Accuracy 84.375
F1 0.8333333333333334
[[29  3]
 [ 7 25]]
Accuracy 87.5
F1 0.8857142857142857
[[25  7]
 [ 1 31]]
--------------Average------------
Avg Accuracy 86.875
Avg F1 0.8770265555340181
Avg Confusion [[0. 0.]
 [0. 0.]]


### Apply to Target

In [32]:
class TargetDataset(Dataset):
    def __init__(self, root_dir=None, transform=None):
        path = os.path.join(root_dir,'ylabels.csv')
        self.ylabels = pd.read_csv(path)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.ylabels)

    def __getitem__(self, idx):
        seq_len = 5
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        label = 0
        user = self.ylabels.iloc[idx]['Participant ID']
        
        img_arr = np.zeros((5, 1, 64, 64))
        for i in range (5):
            img_name = self.root_dir + 'p_' + str(user) + '-' + str(i) + '.jpg'
            image = io.imread(img_name)
            if self.transform:
                image = self.transform(image)
            img_arr[i][0] = image

        label_1hot = np.zeros(3)
        label_1hot[label] = 1.0
        label_1hot = torch.Tensor(label_1hot)
#         label = torch.LongTensor(np.array([label]))
        sample = {'images': img_arr, 'label': label_1hot}
        return sample

In [51]:
def apply():
    PATH = './TF_Models/best/model-fold-' + str(0) + '.pth'
    encoder = build_encoder()
    decoder = build_decoder()
    trained_net = Net(encoder,decoder)
    trained_net.load_state_dict(torch.load(PATH))
    trained_net.to(device)

    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5), (0.5))
    ])

    train_data_dir = './Target_Prepared_Data/S4/'
    train_dataset = TargetDataset(root_dir=train_data_dir, transform = transformer)
    testloader = torch.utils.data.DataLoader(train_dataset, batch_size=1)
    outputs = []
    for i, data in enumerate(testloader, 1):
        # Get inputs
        inputs = data['images']
        targets = data['label']
        inputs = inputs.float().to(device)
        inputs = torch.reshape(inputs,(5,1,64,64))
        inputs = Variable(inputs)
        out = sliding_window(inputs, trained_net)
#         out = trained_net(inputs)
        output = out.cpu().detach().numpy()
        outputs.append(output)
    return outputs

In [52]:
out=apply()

In [53]:
pd.DataFrame(np.argmax(out, axis=1)).to_csv('results.csv')